<a href="https://colab.research.google.com/github/arkwith7/aSSIST_ML/blob/main/Llama2_fine_tuning_korquad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Llama 2를 KorQuad 데이터셋에 맞게 fine-tuning 하기
### References:
1.   https://www.youtube.com/watch?v=LslC2nKEEGU
2.   https://colab.research.google.com/drive/1JDnGJbxT8fSqwnXY8J-XFo73AtiSuQMe?usp=sharing

In [8]:
!nvidia-smi

Fri Oct 13 07:55:12 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## AutoTrain Advanced
https://github.com/huggingface/autotrain-advanced

In [9]:
!pip install -q autotrain-advanced

## PyTorch 업데이트

In [10]:
!autotrain setup --update-torch

> INFO    Installing latest transformers@main
> INFO    Successfully installed latest transformers
> INFO    Installing latest peft@main
> INFO    Successfully installed latest peft
> INFO    Installing latest diffusers@main
> INFO    Successfully installed latest diffusers
> INFO    Installing latest trl@main
> INFO    Successfully installed latest trl
> INFO    Installing latest xformers
> INFO    Successfully installed latest xformers
> INFO    Installing latest PyTorch
> INFO    Successfully installed latest PyTorch


## Korquad 데이터셋에 맞게 Llama2 Fine-Tuning

- autotrain 설정값: https://github.com/huggingface/autotrain-advanced/blob/f1367b590dfc53d240e9684779991da540590386/src/autotrain/cli/run_llm.py#L21

In [11]:
# Data Augmentation 적용 x
# !autotrain llm --train \
#     --project_name "llama2-korquad-finetuning" \
#     --model "TinyPixel/Llama-2-7B-bf16-sharded" \
#     --data_path "korquad_prompt" \
#     --text_column "text" \
#     --use_peft \
#     --use_int4 \
#     --learning_rate 2e-4 \
#     --train_batch_size 4 \
#     --num_train_epochs 100 \
#     --trainer sft \
#     --model_max_length 256
# Data Augmentation 적용 o
!autotrain llm --train \
    --project_name "llama2-korquad-finetuning-da" \
    --model "TinyPixel/Llama-2-7B-bf16-sharded" \
    --data_path "korquad_prompt_da" \
    --text_column "text" \
    --use_peft \
    --use_int4 \
    --learning_rate 2e-4 \
    --train_batch_size 8 \
    --num_train_epochs 40 \
    --trainer sft \
    --model_max_length 256

> INFO    Running LLM
> INFO    Params: Namespace(version=False, train=True, deploy=False, inference=False, data_path='korquad_prompt_da', train_split='train', valid_split=None, text_column='text', rejected_text_column='rejected', model='TinyPixel/Llama-2-7B-bf16-sharded', learning_rate=0.0002, num_train_epochs=40, train_batch_size=8, warmup_ratio=0.1, gradient_accumulation_steps=1, optimizer='adamw_torch', scheduler='linear', weight_decay=0.0, max_grad_norm=1.0, seed=42, add_eos_token=False, block_size=-1, use_peft=True, lora_r=16, lora_alpha=32, lora_dropout=0.05, logging_steps=-1, project_name='llama2-korquad-finetuning-da', evaluation_strategy='epoch', save_total_limit=1, save_strategy='epoch', auto_find_batch_size=False, fp16=False, push_to_hub=False, use_int8=False, model_max_length=256, repo_id=None, use_int4=True, trainer='sft', target_modules=None, merge_adapter=False, token=None, backend='default', username=None, use_flash_attention_2=False, disable_gradient_checkpointing=Fal

# 학습결과 zip 파일로 압축후 다운로드

In [12]:
import zipfile
import shutil
from google.colab import files

# 압축할 폴더 이름
#folder_name = "llama2-korquad-finetuning"    # Data Augmentation 적용 x
folder_name = "llama2-korquad-finetuning-da"  # Data Augmentation 적용 o

# 생성될 ZIP 파일 이름
#zip_file_name = "llama2-korquad-finetuning.zip"  # Data Augmentation 적용 x
zip_file_name = "llama2-korquad-finetuning-da.zip" # Data Augmentation 적용 o

# 폴더를 ZIP 파일로 압축
shutil.make_archive(zip_file_name[:-4], 'zip', folder_name)

# ZIP 파일을 로컬로 다운로드
files.download(zip_file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>